In [1]:

from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.datasets import cifar10
import numpy as np


Using TensorFlow backend.
/home/menion/anaconda3/envs/ai/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:

LOG_DIR = "logs/keras_autoencoders"
KERAS_WEIGHTS = "keras_weights/weights"


In [3]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[np.where(y_train==1)[0],:,:,:]
x_test = x_test[np.where(y_test==1)[0],:,:,:]


In [4]:

# Normalize and COnvert the data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Add Gaussian Noise
x_train_n = x_train + 0.5 * np.random.normal(loc=0.0, scale=0.4, size=x_train.shape)
x_test_n = x_test + 0.5 * np.random.normal(loc=0.0, scale=0.4, size=x_test.shape)

x_train_n = np.clip(x_train_n, 0., 1.)
x_test_n = np.clip(x_test_n, 0., 1.)


In [5]:

# 32X32 pixel x 3 RGB values
inp_img = Input(shape=(32, 32, 3))

# Model
img = Conv2D(32, (3, 3), activation='relu', padding='same')(inp_img)
img = MaxPooling2D((2, 2), padding='same')(img)
img = Conv2D(32, (3, 3), activation='relu', padding='same')(img)
img = UpSampling2D((2, 2))(img)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(img)

autoencoder = Model(inp_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')


In [6]:

tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, 
                         write_graph=True, write_images=True)

model_saver = ModelCheckpoint(filepath=KERAS_WEIGHTS,verbose=0, period=2)

autoencoder.fit(x_train_n, x_train, 
                epochs=10, 
                batch_size=64,
                shuffle=True, 
                validation_data=(x_test_n, x_test),
                callbacks=[tensorboard, model_saver])


Train on 5000 samples, validate on 1000 samples
Epoch 1/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.6443 - val_loss: 0.5982
Epoch 2/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.5888 - val_loss: 0.5821
Epoch 3/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.5711 - val_loss: 0.5646
Epoch 4/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.5625 - val_loss: 0.5516
Epoch 5/10
5000/5000 [==============================] - 20s 4ms/step - loss: 0.5575 - val_loss: 0.5542
Epoch 6/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.5541 - val_loss: 0.5517
Epoch 7/10
5000/5000 [==============================] - 18s 4ms/step - loss: 0.5515 - val_loss: 0.5476
Epoch 8/10
5000/5000 [==============================] - 19s 4ms/step - loss: 0.5492 - val_loss: 0.5502
Epoch 9/10
5000/5000 [==============================] - 18s 4ms/step - loss: 0.5484 - val_loss: 0.5484
Epoch 10/10
5000/5000 [==